In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import sympy as sym
from pylab import *
from scipy.ndimage import measurements
from copy import copy, deepcopy

In [ ]:
N = 30 #sets the grid size 
x = np.random.rand(N,N)

In [ ]:
for i in range(0,N):
    for j in range(0,N):
        if x[i,j] < 0.50:
            x[i,j]=1
        else:
            x[i,j]=-1 #Generates a T= inf state

In [ ]:
plt.imshow(x)
#pt.colorbar() #Visualising the state

<center> Thus we generate $x$ which is a NxN random array of 1 and -1  

In [ ]:
# Gives us the energy associated with single spin
def spin(x,i,j):
    if i==N-1 and j==N-1:
        return x[i,j]*(x[i,j-1] + x[0,j] + x[i,0] + x[i-1,j]) 
    elif i==N-1:
        return x[i,j]*(x[i,j-1] + x[0,j] + x[i,j+1] + x[i-1,j])
    elif j==N-1:
        return x[i,j]*(x[i,j-1] + x[i+1,j] + x[i,0] + x[i-1,j])
    else:
        return x[i,j]*(x[i,j-1] + x[i+1,j] + x[i,j+1] + x[i-1,j]) 

In [ ]:
#Gives us the energy of the state
def Energy(array):
    Enn=0
    for i in range(0,N):
        for j in range(0,N): 
            Enn = Enn - spin(array,i,j)
    return (1/2)*Enn
    

In [ ]:
#Gives us the average cluster size of +1 spins
def avg_size(arr):
    a = deepcopy(arr)
    a[a==-1]=0
    lw, num = measurements.label(a)
    area = measurements.sum(a, lw, index=arange(lw.max() + 1))
    return area[1:].mean()

<center> spin(i,j) gives us the associated energy of each spin; and Energy(x) gives us the hamiltonian of state and the mag(array) gives us the magnetisation of the state</center>

In [ ]:
def choose_neighbour(p,q,n):
    if p==N-1 and q== N-1:
        if n==1:
            return p,q-1
        elif n==2:
            return 0,q
        elif n==3:
            return p,0
        elif n==4:
            return p-1,q
    elif p==N-1:
        if n==1:
            return p,q-1
        elif n==2:
            return 0,q
        elif n==3:
            return p,q+1
        elif n==4:
            return p-1,q
    elif q==N-1:
        if n==1:
            return p,q-1
        elif n==2:
            return p+1,q
        elif n==3:
            return p,0
        elif n==4:
            return p-1,q
    else:
        if n==1:
            return p,q-1
        elif n==2:
            return p+1,q
        elif n==3:
            return p,q+1
        elif n==4:
            return p-1,q

In [ ]:
#This function implements the metropolis algorithm and calculates the needed quantities
def equilibrate(y,T):
    H=Energy(y)
    E_calc=[H]
    sizes=[]
    for i in range(5*10**4):
        p = np.random.randint(0,N)
        q = np.random.randint(0,N)
        n=np.random.choice([1,2,3,4])
        r,s = choose_neighbour(p,q,n)
        dSp= -y[p,q] + y[r,s]
        dE = dSp * (spin(y,r,s)/y[r,s] - spin(y,p,q)/y[p,q] + dSp)
        if dE <=0 :
            y[p,q] , y[r,s] = y[r,s] , y[p,q]
            E_new = H +dE
        else:
            temp=np.random.rand()
            prob = np.exp(-(dE)/T)
            if temp<prob:
                y[p,q] , y[r,s] = y[r,s] , y[p,q]
                E_new = H+ dE
            else:
                E_new = H
        E_calc.append(E_new)
        H = E_new
        size=avg_size(y)
        sizes.append(size)

    E_calc = np.array(E_calc)
    return y, sizes


In [ ]:
## The Following cells calculates the cluster sizes and then fits the log-log plot and then shows the plot.
sizess=[]
for i in range(15):
    x = np.random.rand(N,N)
    for i in range(0,N):
        for j in range(0,N):
            if x[i,j] < 0.6:
                x[i,j]=1
            else:
                x[i,j]=-1
    x,sizes = equilibrate(x,0.6)
    sizess.append(sizes)

In [ ]:
npsz=np.array(sizess)
sizes=0
for i in range(15):
    sizes += npsz[i]
sizes= np.array(sizes) / len(npsz)

In [ ]:
mc_steps=np.arange(0,5*10**4)

In [ ]:
def gro(x,a,b):
    return a+x*b

In [ ]:
y_data = np.array(np.log(sizes)[10**2:],dtype=float)
x_data = np.array(np.log(mc_steps)[10**2:],dtype = float)
popt, pcov = curve_fit(gro, x_data, y_data, maxfev=10**6)

In [ ]:
plt.plot(np.log(mc_steps)[10**3:], np.log(sizes)[10**3:],'.',markersize=5,label='Simulation Data')
plt.plot(x_data[6*10**2:],(popt[0]+x_data*popt[1])[6*10**2:],label='$y=2.83+0.20x$')
plt.xlabel('Monte Carlo Steps \n towards equilibrium $k_BT=0.6$')
plt.ylabel('Average Positive Spin Cluster Size')
plt.title('Log-Log Plot')
plt.legend()

In [ ]:
#Defining the two-point correlation function
def g(x,r):
    sum = 0
    for k in range(10**5):
        i = np.random.randint(0,N)
        j= np.random.randint(0,N)
        sum += x[i,j]*(x[i-r,j])
    return sum/(10**5)

In [ ]:
#The following parts gets the two-point-correlation values for different temperatures,
#fits it to get the correlation lengths at different temperatures then plots it
g_T_s=[]
r_s= [1,2,3,4,5,6,7,8,9,10]
T_s = np.arange(2.2,4,0.05)
for T in T_s:
    y=equilibrate(x,T)
    g_ss=[]
    for i in range(0,10):
        g_s=[]
        for r in r_s:
            g_r = g(y,r)
            g_s.append(g_r)
        g_ss.append(g_s)
    g_ss_np=np.array(g_ss)
    g_s_f=[]
    temp=0
    for i in range(len(g_ss)):
        temp+=g_ss_np[i]
    g_s_f=np.array(temp)/len(g_ss)
    g_T_s.append(g_s_f)

In [ ]:
def func(x,a,b):
    return a*np.exp(-x/b)

In [ ]:
x_data = np.array(r_s,dtype=float)
corr_lengths =[]
hm=[]
for i in range(len(g_T_s)):
    y_data = np.array(g_T_s[i],dtype = float)
    popt, pcov = curve_fit(func, x_data , y_data , maxfev=10000)
    corr_lengths.append(popt[1])
    hm.append(popt)

In [ ]:
plt.plot(T_s[7:],corr_lengths[7:],'o--')
plt.xlabel('$k_BT$')
plt.ylabel('Correlation Length')

In [ ]:
def func2(x,b):
    return abs(x-2.35)**(-b)

In [ ]:
popt, pcov = curve_fit(func2,T_s[7:],corr_lengths[7:], maxfev=100000)

In [ ]:
fit_corr=abs(T_s-2.3)**(-popt[0])

In [ ]:
plt.plot(T_s[7:],(np.array(corr_lengths)[7:]),'o--',label='Computed Data')
plt.plot(T_s[6:], fit_corr[6:],label='Fitted plot, $f(T)=|T-2.3|^{-0.55}$')
plt.legend()
plt.xlabel('$K_BT$')
plt.ylabel('Correlation Length')

In [ ]:
#After equilibration we use this to get the values of the observables
def calculate(y,T):
    H=Energy(y)
    E_calc=[H]
    for i in range(10**6):
        p = np.random.randint(0,N)
        q = np.random.randint(0,N)
        n=np.random.choice([1,2,3,4])
        r,s = choose_neighbour(p,q,n)
        #if y[p,q] != y[r,s]:
        dSp= -y[p,q] + y[r,s]
        dE = dSp * (spin(y,r,s)/y[r,s] - spin(y,p,q)/y[p,q] + dSp)
        if dE <=0 :
            y[p,q] , y[r,s] = y[r,s] , y[p,q]
            E_new = H +dE
        else:
            temp=np.random.rand()
            prob = np.exp(-(dE)/T)
            if temp<prob:
                y[p,q] , y[r,s] = y[r,s] , y[p,q]
                E_new = H+ dE
            else:
                E_new = H
        E_calc.append(E_new)
        H = E_new
    E_calc = np.array(E_calc)
    
    E_avg= E_calc.mean()
    C= E_calc.var()/T**2
    return [E_avg , C]


In [ ]:
temp=np.arange(0.1,4,0.1) #Generating a set of temperatures 

In [ ]:
E_s=[]
C_s=[]
for T in temp:
    y=np.array(x)
    y= equilibrate(y,T)
    E_avg, C =calculate(y,T)
    E_s.append(E_avg)
    C_s.append(C)
#Running the metropolis algorithm on the state at different temperatures and plotting

In [ ]:
plt.plot(temp,E_s, 'o--')
plt.xlabel('$k_bT$')
plt.ylabel('$E_{avg}$')

In [ ]:
plt.plot(temp, C_s, 'o--')
plt.grid()
plt.xlabel('$k_bT$')
plt.ylabel('Heat Capacity')